In [105]:
import pandas as pd
import numpy as np

In [106]:
# define variables
BATTERY_CAPACITY = 580 # maximum capacity MWH
DISCHARGE_EFFICIENCY = 0.9
CHARGE_EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
MAX_CAPACITY = 300

In [107]:
# this function takes dataframe name as output of charge/discharge decision
# and preprocess it to match the generate capacity function
# assumes the csv files are in the same folder

def process_csv(df_name):
    df = pd.read_csv(df_name)
    # df = df.iloc[:,[1, 2, 4]] 'uncomment for label.csv'
    # df.columns = ['Time', 'Price', 'Status'] 'uncomment for label.csv'
    df['Status'].replace('Disharge', 'Discharge', inplace=True)
    df['Status'].replace(0, 'Do Nothing', inplace=True)
    df['Status'].replace(1, 'Charge', inplace=True)
    df['Status'].replace(-1, 'Discharge', inplace=True)
    df = df.dropna().reset_index(drop=True)
    return df

In [108]:
# generate_capacity calculates all the columns needed to calculate revenue
# and takes dataframe with columns ['Time', 'Price', 'Status'] as input

def generate_capacity(df):
    
    # initialises variable
    raw_power = 0
    opening_capacity = 0
    closing_capacity = 0
    market_dispatch = 0

    raw_power_s = pd.Series(dtype = float)
    opening_capacity_s = pd.Series(dtype = float)
    closing_capacity_s = pd.Series(dtype = float)
    market_dispatch_s = pd.Series(dtype = float)
    
    status = df['Status']
    for i in df.index:
        # print(i)
        if i == 0: # set opening capacity
            opening_capacity_s.loc[i] = 0
        else: # set opening capacity to previous closing capacity
            opening_capacity = closing_capacity_s.loc[i-1]
            opening_capacity_s.loc[i] = closing_capacity_s.loc[i-1]
        if status[i] == 'Charge':
            raw_power = -min(MAX_CAPACITY, ((int(BATTERY_CAPACITY) - float(opening_capacity_s[i]))/CHARGE_EFFICIENCY) * 2) # 300 is the minimum capacity
            market_dispatch = raw_power / 2
            closing_capacity = max(0, min(opening_capacity_s[i] - market_dispatch * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            raw_power_s.loc[i] = raw_power
            market_dispatch_s.loc[i] = market_dispatch
            closing_capacity_s.loc[i] = closing_capacity
        elif status[i] == 'Discharge':
            raw_power = min(MAX_CAPACITY, (float(opening_capacity_s[i]) * 2)) ##/ DISCHARGE_EFFICIENCY)
            market_dispatch = raw_power/2 * DISCHARGE_EFFICIENCY
            closing_capacity = max(0, min(opening_capacity_s[i] - (market_dispatch * 1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            market_dispatch_s.loc[i] = market_dispatch
            closing_capacity_s.loc[i] = closing_capacity
            raw_power_s.loc[i] = raw_power
        else:
            raw_power_s.loc[i] = 0
            market_dispatch_s.loc[i] = 0
            closing_capacity_s.loc[i] = closing_capacity
    
    df['raw power'] = raw_power_s
    df['market dispatch'] = market_dispatch_s
    df['opening capacity'] = opening_capacity_s
    df['closing capacity'] = closing_capacity_s

In [109]:
# calculate_revenue calculates revenue from 2018-2021

def calculate_revenue(df):
    revenue_s = pd.Series(dtype = float)
    for i in df.index:
        if df['market dispatch'].loc[i] < 0:
            revenue_s.loc[i] = df['Price'].loc[i] * df['market dispatch'].loc[i] * (1 / MARGINAL_LOSS_FACTOR)
        else:
            revenue_s.loc[i] = df['Price'].loc[i] * df['market dispatch'].loc[i] * (MARGINAL_LOSS_FACTOR)
            
    df['revenue'] = revenue_s
    return sum(df['revenue'])

## Calculate revenue from data
-- for testing purposes only --

In [15]:
rev = process_csv('rev.csv')
rev

,Unnamed: 0,Time,Price,MA,Status,Dispatch,_merge
0,5,2018-01-01 10:30:00,71.13,70.582571,Discharge,-150.0,left_only
1,6,2018-01-01 11:00:00,74.86,70.514571,Discharge,-150.0,left_only
2,7,2018-01-01 11:30:00,72.45,70.699429,Discharge,-150.0,left_only
3,8,2018-01-01 12:00:00,66.51,71.032571,Charge,135.0,left_only
4,10,2018-01-01 16:00:00,85.27,71.699714,Discharge,135.0,left_only
...,...,...,...,...,...,...,...
28402,29968,2021-12-08 11:30:00,18.56,51.996000,Charge,135.0,left_only
28403,29969,2021-12-08 12:00:00,17.39,54.709143,Charge,135.0,left_only
28404,29970,2021-12-08 12:30:00,5.71,57.652000,Charge,135.0,left_only
28405,29971,2021-12-08 13:00:00,8.95,60.270571,Charge,135.0,left_only


In [16]:
generate_capacity(rev)

In [17]:
calculate_revenue(rev)

126377425.53897737

### 1

In [92]:
zig10 = process_csv('zig10.csv')
zig10

,Unnamed: 0,Time,Price,Status
0,0,2018-01-01 00:00:00,90.43,Do Nothing
1,1,2018-01-01 00:30:00,92.46,Do Nothing
2,2,2018-01-01 01:00:00,87.62,Do Nothing
3,3,2018-01-01 01:30:00,73.08,Do Nothing
4,4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...,...
63452,63452,2021-08-14 22:00:00,49.93,Discharge
63453,63453,2021-08-14 22:30:00,62.86,Discharge
63454,63454,2021-08-14 23:00:00,32.26,Discharge
63455,63455,2021-08-14 23:30:00,25.10,Do Nothing


In [ ]:
generate_capacity(zig10)

In [ ]:
calculate_revenue(zig10)

### 2

In [67]:
zig5 = process_csv('zig5.csv')
zig5

,Unnamed: 0,Time,Price,Status
0,0,2018-01-01 00:00:00,90.43,Do Nothing
1,1,2018-01-01 00:30:00,92.46,Do Nothing
2,2,2018-01-01 01:00:00,87.62,Do Nothing
3,3,2018-01-01 01:30:00,73.08,Do Nothing
4,4,2018-01-01 02:00:00,70.18,Charge
...,...,...,...,...
63452,63452,2021-08-14 22:00:00,49.93,Discharge
63453,63453,2021-08-14 22:30:00,62.86,Discharge
63454,63454,2021-08-14 23:00:00,32.26,Discharge
63455,63455,2021-08-14 23:30:00,25.10,Discharge


In [69]:
generate_capacity(zig5)

In [ ]:
calculate_revenue(zig5)

### 3

In [30]:
zigMA = process_csv('zigMA.csv')
zigMA

,Unnamed: 0,Time,Price,Status
0,0,2018-01-01 00:00:00,90.43,Do Nothing
1,1,2018-01-01 00:30:00,92.46,Do Nothing
2,2,2018-01-01 01:00:00,87.62,Do Nothing
3,3,2018-01-01 01:30:00,73.08,Do Nothing
4,4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...,...
63452,63452,2021-08-14 22:00:00,49.93,Do Nothing
63453,63453,2021-08-14 22:30:00,62.86,Do Nothing
63454,63454,2021-08-14 23:00:00,32.26,Do Nothing
63455,63455,2021-08-14 23:30:00,25.10,Do Nothing


In [31]:
generate_capacity(zigMA)

In [32]:
calculate_revenue(zigMA)

97421242.97958785

### 4

In [135]:
label = process_csv('label.csv')
label.isna().sum() # check nan values

Time      0
Price     0
Status    0
dtype: int64

In [136]:
generate_capacity(label)

In [137]:
calculate_revenue(label)

24262216.062602937

### 4.1 (sample data to check)

In [127]:
label_sample = label.iloc[0:30]
label_sample = label_sample.dropna().reset_index(drop=True)

In [128]:
generate_capacity(label_sample)

In [129]:
label_sample

,Time,Price,Status,raw power,market dispatch,opening capacity,closing capacity,revenue
0,2018-01-01 00:00:00,90.43,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
1,2018-01-01 00:30:00,92.46,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
2,2018-01-01 01:00:00,87.62,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
3,2018-01-01 01:30:00,73.08,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
4,2018-01-01 02:00:00,70.18,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
5,2018-01-01 02:30:00,67.43,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
6,2018-01-01 03:00:00,66.31,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
7,2018-01-01 03:30:00,67.72,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
8,2018-01-01 04:00:00,65.50,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000
9,2018-01-01 04:30:00,64.50,Do Nothing,0.000000,0.000000,0.0,0.0,0.000000


## 5

In [141]:
count = process_csv('count.csv')
count.isna().sum() # check nan values

Time      0
Price     0
Status    0
dtype: int64

In [139]:
count

,Time,Price,Status
0,2018-01-01 00:00:00,90.43,Do Nothing
1,2018-01-01 00:30:00,92.46,Do Nothing
2,2018-01-01 01:00:00,87.62,Do Nothing
3,2018-01-01 01:30:00,73.08,Do Nothing
4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...
61291,2021-06-30 21:30:00,65.79,Charge
61292,2021-06-30 22:00:00,92.52,Charge
61293,2021-06-30 22:30:00,67.46,Charge
61294,2021-06-30 23:00:00,60.16,Do Nothing


In [142]:
generate_capacity(count)

In [143]:
calculate_revenue(count)

17074123.446428295

## 6

In [147]:
MA15 = process_csv('MA15-ZZ102.csv')
MA15

,Unnamed: 0,Time,Price,Status
0,0,2018-01-01 00:00:00,90.43,Do Nothing
1,1,2018-01-01 00:30:00,92.46,Do Nothing
2,2,2018-01-01 01:00:00,87.62,Do Nothing
3,3,2018-01-01 01:30:00,73.08,Do Nothing
4,4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...,...
63452,63452,2021-12-08 21:30:00,81.98,Do Nothing
63453,63453,2021-12-08 22:00:00,71.77,Do Nothing
63454,63454,2021-12-08 22:30:00,100.04,Do Nothing
63455,63455,2021-12-08 23:00:00,81.22,Do Nothing


In [ ]:
generate_capacity(MA15)

In [ ]:
calculate_revenue(MA15)

## 6

In [210]:
MA2017 = process_csv('check3rev.csv')
MA2017

,Time,Price,Status
0,2018-01-01 00:00:00,90.43,Do Nothing
1,2018-01-01 00:30:00,92.46,Do Nothing
2,2018-01-01 01:00:00,87.62,Do Nothing
3,2018-01-01 01:30:00,73.08,Do Nothing
4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...
63452,2021-12-08 21:30:00,81.98,Do Nothing
63453,2021-12-08 22:00:00,71.77,Do Nothing
63454,2021-12-08 22:30:00,100.04,Do Nothing
63455,2021-12-08 23:00:00,81.22,Do Nothing


In [211]:
MA2017.to_csv('check3rev.csv')

In [212]:
MA2017

,Time,Price,Status
0,2018-01-01 00:00:00,90.43,Do Nothing
1,2018-01-01 00:30:00,92.46,Do Nothing
2,2018-01-01 01:00:00,87.62,Do Nothing
3,2018-01-01 01:30:00,73.08,Do Nothing
4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...
63452,2021-12-08 21:30:00,81.98,Do Nothing
63453,2021-12-08 22:00:00,71.77,Do Nothing
63454,2021-12-08 22:30:00,100.04,Do Nothing
63455,2021-12-08 23:00:00,81.22,Do Nothing


In [213]:
MA2017_1 = MA2017.loc[(MA2017['Time'] >= '2020-07-17 00:30:00') & (MA2017['Time'] <= '2020-07-18 00:00:00')].reset_index(drop=True)

In [214]:
MA2017_1

,Time,Price,Status
0,2020-07-17 00:30:00,75.51,Discharge
1,2020-07-17 01:00:00,73.98,Do Nothing
2,2020-07-17 01:30:00,75.57,Do Nothing
3,2020-07-17 02:00:00,71.94,Do Nothing
4,2020-07-17 02:30:00,74.10,Do Nothing
5,2020-07-17 03:00:00,67.36,Charge
6,2020-07-17 03:30:00,58.04,Charge
7,2020-07-17 04:00:00,51.85,Charge
8,2020-07-17 04:30:00,74.53,Charge
9,2020-07-17 05:00:00,67.32,Charge


In [215]:
generate_capacity(MA2017_1)

In [216]:
calculate_revenue(MA2017_1)

43355.217125731586

In [217]:
pd.set_option('display.max_rows', MA2017.shape[0]+1)

## 7

In [6]:
MAdf_df2 = process_csv('MAdf_df2.csv')

In [7]:
MAdf_df2

,Unnamed: 0,Time,Price,Status
0,0,2018-01-01 00:00:00,90.43,Do Nothing
1,1,2018-01-01 00:00:00,90.43,Do Nothing
2,2,2018-01-01 00:30:00,92.46,Do Nothing
3,3,2018-01-01 01:00:00,87.62,Do Nothing
4,4,2018-01-01 01:30:00,73.08,Do Nothing
...,...,...,...,...
105697,105697,2021-08-14 22:30:00,62.86,Do Nothing
105698,105698,2021-08-14 22:30:00,62.86,Do Nothing
105699,105699,2021-08-14 23:00:00,32.26,Do Nothing
105700,105700,2021-08-14 23:30:00,25.10,Do Nothing


In [8]:
generate_capacity(MAdf_df2)

In [10]:
calculate_revenue(MAdf_df2)

93527563.81686416

## 8

In [12]:
MAdf_df3 = process_csv('MAdf_df3.csv')

In [13]:
generate_capacity(MAdf_df3)

In [14]:
calculate_revenue(MAdf_df3)

78730337.5804051

## 9

In [6]:
final = process_csv('adrenaline.csv')

In [7]:
final

,Time,Price,Status,Dispatch,mrf,Revenue
0,2018-01-01 00:00:00,90.43,Do Nothing,0.0,0.991,0.0
1,2018-01-01 00:30:00,92.46,Do Nothing,0.0,0.991,0.0
2,2018-01-01 01:00:00,87.62,Do Nothing,0.0,0.991,0.0
3,2018-01-01 01:30:00,73.08,Do Nothing,0.0,0.991,0.0
4,2018-01-01 02:00:00,70.18,Do Nothing,0.0,0.991,0.0
...,...,...,...,...,...,...
36397,2021-12-08 21:30:00,81.98,Do Nothing,0.0,0.991,0.0
36398,2021-12-08 22:00:00,71.77,Do Nothing,0.0,0.991,0.0
36399,2021-12-08 22:30:00,100.04,Do Nothing,0.0,0.991,0.0
36400,2021-12-08 23:00:00,81.22,Do Nothing,0.0,0.991,0.0


In [8]:
generate_capacity(final)

In [9]:
calculate_revenue(final)

119307884.10011975

In [11]:
final.head(50)

,Time,Price,Status,Dispatch,mrf,Revenue,raw power,market dispatch,opening capacity,closing capacity,revenue
0,2018-01-01 00:00:00,90.43,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,2018-01-01 00:30:00,92.46,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2,2018-01-01 01:00:00,87.62,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
3,2018-01-01 01:30:00,73.08,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
4,2018-01-01 02:00:00,70.18,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
5,2018-01-01 02:30:00,67.43,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
6,2018-01-01 03:00:00,66.31,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
7,2018-01-01 03:30:00,67.72,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
8,2018-01-01 04:00:00,65.50,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000
9,2018-01-01 04:30:00,64.50,Do Nothing,0.000000,0.991,0.000000,0.000000,0.000000,0.0,0.0,0.000000


## 10

In [18]:
inter = process_csv('inter.csv')

In [19]:
inter

,Unnamed: 0,Time,Price,Status
0,0,2018-01-01 00:00:00,90.43,Do Nothing
1,1,2018-01-01 00:30:00,92.46,Do Nothing
2,2,2018-01-01 01:00:00,87.62,Do Nothing
3,3,2018-01-01 01:30:00,73.08,Do Nothing
4,4,2018-01-01 02:00:00,70.18,Do Nothing
...,...,...,...,...
63452,63452,2021-12-08 21:30:00,81.98,Do Nothing
63453,63453,2021-12-08 22:00:00,71.77,Do Nothing
63454,63454,2021-12-08 22:30:00,100.04,Do Nothing
63455,63455,2021-12-08 23:00:00,81.22,Do Nothing


In [20]:
generate_capacity(inter)

In [21]:
calculate_revenue(inter)

33347095.390472963

## 11

In [22]:
inter = process_csv('Z.csv')

In [23]:
generate_capacity(inter)

In [24]:
calculate_revenue(inter)

-79584581.9375124

In [26]:
inter.head(50)

,Time,Price,Status,raw power,market dispatch,opening capacity,closing capacity,revenue
0,2018-01-01 00:00:00,90.43,Do Nothing,0.0,0.0,0.0,0.0,0.000000
1,2018-01-01 00:30:00,92.46,Do Nothing,0.0,0.0,0.0,0.0,0.000000
2,2018-01-01 01:00:00,87.62,Do Nothing,0.0,0.0,0.0,0.0,0.000000
3,2018-01-01 01:30:00,73.08,Do Nothing,0.0,0.0,0.0,0.0,0.000000
4,2018-01-01 02:00:00,70.18,Do Nothing,0.0,0.0,0.0,0.0,0.000000
5,2018-01-01 02:30:00,67.43,Do Nothing,0.0,0.0,0.0,0.0,0.000000
6,2018-01-01 03:00:00,66.31,Do Nothing,0.0,0.0,0.0,0.0,0.000000
7,2018-01-01 03:30:00,67.72,Do Nothing,0.0,0.0,0.0,0.0,0.000000
8,2018-01-01 04:00:00,65.50,Do Nothing,0.0,0.0,0.0,0.0,0.000000
9,2018-01-01 04:30:00,64.50,Do Nothing,0.0,0.0,0.0,0.0,0.000000


## 12

In [27]:
old = process_csv('OldZ.csv')

In [28]:
generate_capacity(old)

In [29]:
calculate_revenue(old)

46330853.385665014

In [6]:
a = pd.read_excel('Victoria_data.xlsx')

In [8]:
a.head(50)

,Time (UTC+10),period,Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Revenue,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2018-01-01 00:30:00,1,92.46,0.000000,0.000000,0,0,0.000000,NaN,Period,Revenue
1,2018-01-01 01:00:00,2,87.62,0.000000,0.000000,0,0,0.000000,NaN,Training,1.19778e+08
2,2018-01-01 01:30:00,3,73.08,0.000000,0.000000,0,0,0.000000,NaN,Testing,5.78041e+06
3,2018-01-01 02:00:00,4,70.18,0.000000,0.000000,0,0,0.000000,NaN,Total,1.25558e+08
4,2018-01-01 02:30:00,5,67.43,0.000000,0.000000,0,0,0.000000,NaN,NaN,NaN
5,2018-01-01 03:00:00,6,66.31,0.000000,0.000000,0,0,0.000000,NaN,NaN,NaN
6,2018-01-01 03:30:00,7,67.72,0.000000,0.000000,0,0,0.000000,NaN,NaN,NaN
7,2018-01-01 04:00:00,8,65.50,0.000000,0.000000,0,0,0.000000,NaN,NaN,NaN
8,2018-01-01 04:30:00,9,64.50,-300.000000,-150.000000,0,135,-9762.865792,NaN,NaN,NaN
9,2018-01-01 05:00:00,10,65.41,0.000000,0.000000,0,0,0.000000,NaN,NaN,NaN
